In [28]:
import pandas as pd
import numpy as np
import json

df = pd.read_json('data1.txt', lines=True)
df.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,quoted_status_permalink,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
0,NaN,None,2018-05-28 07:25:11,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,NaN,0,0,False,{'created_at': 'Sun May 27 15:37:36 +0000 2018...,"<a href=""http://twitter.com/download/iphone"" r...",RT @akchn69: Python中型船で一番使いやすいらしいから長く乗ることになりそう,2018-05-28 07:25:11.959,False,"{'id': 581906827, 'id_str': '581906827', 'name..."
1,NaN,None,2018-05-28 07:25:21,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1001001180003225600, 'id_str...",NaN,0,False,low,...,NaN,0,0,False,{'created_at': 'Mon May 28 07:24:13 +0000 2018...,"<a href=""http://twitter.com/download/android"" ...",RT @LaterCapitalism: I'm learning Python https...,2018-05-28 07:25:21.476,False,"{'id': 1575803474, 'id_str': '1575803474', 'na..."
2,NaN,None,2018-05-28 07:25:31,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,NaN,0,0,False,{'created_at': 'Sat May 26 10:30:27 +0000 2018...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @fchantrel: Et si vous êtes allergique au p...,2018-05-28 07:25:31.282,False,"{'id': 354923441, 'id_str': '354923441', 'name..."
3,NaN,None,2018-05-28 07:25:38,NaN,"{'hashtags': [{'text': 'BigData', 'indices': [...",NaN,NaN,0,False,low,...,NaN,0,0,False,{'created_at': 'Mon May 28 03:28:03 +0000 2018...,"<a href=""http://google.com"" rel=""nofollow"">pme...",RT @gp_pulipaka: Choosing a Machine Learning F...,2018-05-28 07:25:38.913,False,"{'id': 15324359, 'id_str': '15324359', 'name':..."
4,NaN,None,2018-05-28 07:25:43,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1001001180003225600, 'id_str...",NaN,0,False,low,...,NaN,0,0,False,{'created_at': 'Mon May 28 07:24:13 +0000 2018...,"<a href=""http://twitter.com/download/android"" ...",RT @LaterCapitalism: I'm learning Python https...,2018-05-28 07:25:43.045,False,"{'id': 577889339, 'id_str': '577889339', 'name..."


In [26]:
users = pd.DataFrame.from_records(df["user"].values)
users.head()

,contributors_enabled,created_at,default_profile,default_profile_image,description,favourites_count,follow_request_sent,followers_count,following,friends_count,...,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,time_zone,translator_type,url,utc_offset,verified
0,False,Wed May 16 15:01:53 +0000 2012,False,False,"ロリコンで虹専の救いようのない21歳です！現在留学でKansas,wichita在住。最近サ...",23071,None,467,None,439,...,333333,True,False,nazeru2,20547,None,none,https://inara.cz/cmdr/103080/,None,False
1,False,Sun Jul 07 18:45:00 +0000 2013,True,False,None,946,None,63,None,434,...,333333,True,False,CONGOBILL1,1724,None,none,None,None,False
2,False,Sun Aug 14 14:46:09 +0000 2011,False,False,Curious lead software & web #developer #Java #...,190,None,111,None,192,...,000000,False,False,AlbanBerthout,355,None,none,None,None,False
3,False,Sat Jul 05 08:56:50 +0000 2008,False,False,Passionné de technologie. #CTO/#COO/#coachagil...,7084,None,14770,None,7102,...,000000,False,False,pmedina,68095,None,none,http://fr.linkedin.com/in/pierremedina,None,False
4,False,Sat May 12 09:09:39 +0000 2012,False,False,Little Monsłer /《La historia os condenará y no...,158613,None,1099,None,899,...,333333,True,False,alexcorts22,98597,None,regular,http://ladygaga.com,None,False


In [75]:
from pandas.io.json import json_normalize
import ast

pd.read_json("\n".join(df["retweeted_status"].apply(ast.literal_eval).values), lines=True)

ValueError: Expected object or value

In [44]:
entities = pd.DataFrame.from_records(df["entities"].values)
entities.head()

,hashtags,media,symbols,urls,user_mentions
0,[],NaN,[],[],"[{'screen_name': 'akchn69', 'name': 'R053🐈Sp@c..."
1,[],"[{'id': 1001001180003225600, 'id_str': '100100...",[],[],"[{'screen_name': 'LaterCapitalism', 'name': 'F..."
2,[],NaN,[],[],"[{'screen_name': 'fchantrel', 'name': 'fchantr..."
3,"[{'text': 'BigData', 'indices': [56, 64]}, {'t...",NaN,[],[],"[{'screen_name': 'gp_pulipaka', 'name': 'Dr. G..."
4,[],"[{'id': 1001001180003225600, 'id_str': '100100...",[],[],"[{'screen_name': 'LaterCapitalism', 'name': 'F..."


In [45]:
display(df.columns)
display(users.columns)
display(entities.columns)

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'extended_tweet', 'favorite_count',
       'favorited', 'filter_level', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quote_count', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink',
       'reply_count', 'retweet_count', 'retweeted', 'retweeted_status',
       'source', 'text', 'timestamp_ms', 'truncated', 'user'],
      dtype='object')

Index(['contributors_enabled', 'created_at', 'default_profile',
       'default_profile_image', 'description', 'favourites_count',
       'follow_request_sent', 'followers_count', 'following', 'friends_count',
       'geo_enabled', 'id', 'id_str', 'is_translator', 'lang', 'listed_count',
       'location', 'name', 'notifications', 'profile_background_color',
       'profile_background_image_url', 'profile_background_image_url_https',
       'profile_background_tile', 'profile_banner_url', 'profile_image_url',
       'profile_image_url_https', 'profile_link_color',
       'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_text_color', 'profile_use_background_image', 'protected',
       'screen_name', 'statuses_count', 'time_zone', 'translator_type', 'url',
       'utc_offset', 'verified'],
      dtype='object')

Index(['hashtags', 'media', 'symbols', 'urls', 'user_mentions'], dtype='object')